In [8]:
import os, json, pprint, time
from xml.etree import ElementTree as ET
path = "/data/cm/"

In [2]:
def get_zeitschrift_struktur(path, zs_number):
    print("Zeitschrift:", zs_number, end="\n\n")
    for jahrgang in os.listdir(path+zs_number+"/"+zs_number+"/"):
        print("Jahrgang:", end=" ")
        print(jahrgang)
        print("Hefte:")
        for heft in os.listdir(path+zs_number+"/"+zs_number+"/"+jahrgang+"/"):
            print(heft, end=", ")
            seiten = len(os.listdir(path+zs_number+"/"+zs_number+"/"+jahrgang+"/"+heft))
            print(seiten, "Seiten.")
    print()
    
def xml_to_page(tree):
    # input: page parsed xml tree
    # output: 
    # page --> paragraphs --> lines
    # [[[line1], [line2]], [[line1], [line2]]] etc.
    page = []
    line = []
    par = []
    for node in tree.iter():
        if node.tag.endswith('charParams'):
            line.append(node.text)

        elif node.tag.endswith('line') and line != []:
            par.append(line)
            line = []
        elif node.tag.endswith('par') and par != []:
            page.append(par)
            par = []
    return(page)

def page_blocks_to_text(page_blocks):
    # input: output of xml_to_page
    # output: page as plain text
    text_page = ""
    for par in page_blocks:
        text_par = ""
        for line in par:
            # apparently sometimes line happens to be [None], then...
            if None in line:
                line = [el for el in line if el != None]
            
            text_line = "".join(line).strip()
            text_par += " "
            text_par += text_line
        text_page += text_par
    text_page = text_page.replace("¬ ", "").replace("- ", "").replace("= ", "").replace("^ ", "").strip()
    text_page = text_page.replace(" ;", ";").replace(" ?", "?").replace("•", "")
    text_page = text_page.replace("    ", " ").replace("   ", " ").replace("  ", " ")
    
    return(text_page)

def clean_text(text):
    text = text.replace("¬ ", "").replace("- ", "").replace("= ", "").replace("^ ", "").strip()
    text = text.replace(" ;", ";").replace(" ?", "?").replace("•", "")
    text = text.replace("    ", " ").replace("   ", " ").replace("  ", " ")
    return text

In [10]:
# initialize output dictionary
current_page = []
notadirectory = []
isadirectory = []
start = time.time()
output = []
processed_journals = set()


for zeitschrift in os.listdir(path+"xml/"):
    
    print("Processing now", zeitschrift, end=" ")
    seiten = []

    if zeitschrift not in processed_journals:
    
        for jahrgang in os.listdir(path+"xml/"+zeitschrift+"/"):
        
            try:
        
                for heft in os.listdir(path+"xml/"+zeitschrift+"/"+jahrgang+"/"):
            
                    try:
            
                        for seite in os.listdir(path+"xml/"+zeitschrift+"/"+jahrgang+"/"+heft+"/"):
                            current_page.append(seite)
                            
                            try:
                                seite_nr = os.listdir(path+"xml/"+zeitschrift+"/"+jahrgang+"/"+heft+"/").index(seite)
                                
                                
                                tree = ET.parse(path+"xml/"+zeitschrift+"/"+jahrgang+"/"+heft+"/"+seite)
                                page = xml_to_page(tree)
                            
                                page_text = page_blocks_to_text(page)
                                page_text.replace('"', '‟')
                                
                                #write = open(path+"output/text/")
                                
                                
                                
                                
                                
                                
                            except IsADirectoryError as iade:
                                isadirectory.append(iade)
                    except NotADirectoryError as nade:
                        notadirectory.append(nade)
            except NotADirectoryError as nade:
                notadirectory.append(nade)
        print(zeitschrift, len(notadirectory), len(isadirectory))
        processed_journals.add(zeitschrift)
        
running_time = time.time()-start
print("Done in", running_time)

Processing now 2710055 2710055 0 0
Processing now 9616701 9616701 0 0
Processing now 2580773 2580773 2687 0
Processing now 2895450 2895450 2687 1413
Processing now 2727810 2727810 2687 2270
Processing now 3129675 3129675 2687 2270
Processing now 6492429 6492429 2687 2270
Processing now 2438141 2438141 2687 2270
Processing now 2827798 2827798 2687 2270
Processing now 5438908 5438908 2688 2270
Processing now 4782723 4782723 5639 2270
Processing now 7506414 7506414 5639 2270
Processing now 9054153 9054153 5639 2270
Processing now 9366335 9366335 5639 2270
Processing now 4889892 4889892 5639 2270
Processing now 2425321 2425321 5641 2270
Processing now 6492329 6492329 5641 2272
Processing now 700673 700673 5665 2272
Processing now 4895413 4895413 5665 2272
Processing now 2542917 2542917 5665 2272
Processing now 9583346 9583346 5665 2272
Processing now 2549643 2549643 5665 2272
Processing now 10638461 10638461 5665 2272
Processing now 3139318 3139318 5665 2272
Processing now 9620169 9620169 

KeyboardInterrupt: 

In [ ]:
# serialize to json elasticsearch format
es_index = []
for elem in output:
    index_value = elem['zeitschrift'] + "_" + elem['jahrgang'] + "_" + elem['heft'] + "_" + str(elem['seite'])
    es_index.append(json.dumps({"index": {"_id": index_value, "_index": "cm"}}, ensure_ascii=False))
    es_index.append(json.dumps({"text": elem['text']}, ensure_ascii=False))

with open(path+"output/es/es_data.json", "w", encoding="utf-8") as write_file:
    write_file.write("\n".join(es_index))